<a href="https://colab.research.google.com/github/alexvanhalen/Introduction-to-AI/blob/main/%E6%96%87%E5%AD%97%E8%BD%89%E8%AA%9E%E9%9F%B3_TTS_%E9%9B%B2%E7%AB%AF%E5%8F%AF%E5%9F%B7%E8%A1%8C_Notebook_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#我問：請幫我使用python撰寫一個程式，可以將使用者輸入的文字轉換成語音。

當然可以！下面給你兩種常見做法：

---

# 方案 A：離線合成（`pyttsx3`，不需網路）

**安裝**

```bash
pip install pyttsx3
```

**程式（自動選擇中文語音、輸出成 WAV 並直接播放）**

```python
import pyttsx3
import sys

def pick_chinese_voice(engine):
    # 嘗試依語言標記或名稱挑選中文語音
    for v in engine.getProperty("voices"):
        langs = []
        try:
            # 某些平台 languages 可能是 bytes[]
            langs = [l.decode("utf-8").lower() if isinstance(l, (bytes, bytearray)) else str(l).lower()
                     for l in getattr(v, "languages", [])]
        except Exception:
            pass
        name = v.name.lower() if hasattr(v, "name") else ""
        if any(("zh" in l or "cmn" in l or "zho" in l) for l in langs) or ("chinese" in name or "mandarin" in name or "zh" in name):
            return v.id
    return None

def main():
    text = input("請輸入要轉語音的文字：\n").strip()
    if not text:
        print("未輸入文字，程式結束。")
        sys.exit(0)

    engine = pyttsx3.init()
    # 速度與音量可自行調整
    engine.setProperty("rate", 180)    # 語速
    engine.setProperty("volume", 1.0)  # 音量 0.0 ~ 1.0

    vid = pick_chinese_voice(engine)
    if vid:
        engine.setProperty("voice", vid)
    else:
        print("⚠️ 找不到中文語音，將使用預設語音。你可以安裝系統中文語音後再試。")

    # 直接播放
    engine.say(text)
    engine.runAndWait()

    # 也可另存檔案（取消下行註解）
    # engine.save_to_file(text, "output.wav")
    # engine.runAndWait()
    # print("已儲存為 output.wav")

if __name__ == "__main__":
    main()
```

> 提示
>
> * Windows 常見中文語音：`Microsoft Huihui Desktop`、`Microsoft Yaoyao` 等（需在「設定 → 時間與語言 → 語音」安裝中文語音包）。
> * macOS 於「系統設定 → 錄音與語音/輔助使用 → 語音」下載「Ting-Ting（中文）」等。
> * Linux 可能需要安裝 `espeak` 或 `festival` 等 TTS 後端。

---

# 方案 B：雲端合成（`gTTS`，需網路，產出 MP3）

**安裝**

```bash
pip install gTTS playsound
```

**程式（將輸入文字存成 MP3 並播放）**

```python
from gtts import gTTS
from playsound import playsound
import os

def main():
    text = input("請輸入要轉語音的文字：\n").strip()
    if not text:
        print("未輸入文字，程式結束。")
        return

    # zh-TW（繁體中文），也可用 'zh-CN'（簡體中文）
    tts = gTTS(text=text, lang='zh-TW')
    out = "output.mp3"
    tts.save(out)
    print(f"已儲存為 {out}，開始播放…")
    playsound(out)

if __name__ == "__main__":
    main()
```

---

## 我該選哪個？

* **無網路/需本地即時播放**：選 **方案 A（pyttsx3）**。
* **想要較自然的中文發音、可接受網路需求**：選 **方案 B（gTTS）**。

如果你有特定平台（Windows/macOS/Linux）或想要把它包成函式/GUI，我可以直接幫你調整成你要的版本。


#我問：我要包裝成jupyter notebook的格式，可以直接在雲端執行，請幫我加入必要的文字說明。

# 文字轉語音（TTS）教學與工具 — Jupyter Notebook 版

此 Notebook 提供**可在雲端直接執行**的文字轉語音範例，
採用 **gTTS**（Google Text-to-Speech）將文字轉成 **MP3** 並在 Notebook 內直接播放。

> 建議環境：Google Colab、JupyterLab / Jupyter Notebook（雲端或本機皆可）。

---
## 你會學到什麼？
- 安裝、使用 gTTS 進行中文（或其他語言）語音合成
- 在 Notebook 內直接播放音檔
- 建立**重複使用**的 `text_to_speech_gtts` 函式
-（選配）批次將多段文字輸出成多個音檔
-（說明）為何**離線引擎**（如 `pyttsx3`）在雲端執行時常受到限制

---
## 快速說明
- 若你在**雲端**（如 Colab），**推薦使用 gTTS**（本 Notebook 的主角），因為：
  1) 安裝簡單；2) 能直接在 Notebook 內播放；3) 對中文發音友善。
- **離線引擎**（`pyttsx3`）通常依賴系統 TTS 後端與聲音裝置，
在雲端環境缺少音訊裝置或驅動時，播放/輸出會受限，因此本 Notebook 只提供參考說明。


## 1) 安裝相依套件（gTTS）
執行下方格子來安裝 `gTTS`。在 Colab 或初次執行的雲端環境中，這一步是必要的。

In [ ]:
# 若你使用的是 Colab / 乾淨環境，請先安裝 gTTS
import sys
def _pip_install(pkg):
    print(f"Installing {pkg} ...")
    !{sys.executable} -m pip install -q {pkg}

try:
    import gtts  # type: ignore
    print("gTTS 已可使用。")
except Exception:
    _pip_install("gTTS")
    import gtts
    print("gTTS 安裝完成。")


Installing gTTS ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.7 MB/s eta 0:00:00
gTTS 安裝完成。


## 2) 定義：雲端友善的 `text_to_speech_gtts` 函式
此函式會：
1. 使用 gTTS 合成語音並存成 MP3（預設為 `output.mp3`）
2. 在 Notebook 中以播放器（Audio widget）直接播放

語言碼常用參考：
- `zh-TW`：繁體中文、`zh-CN`：簡體中文、`en`：英文、`ja`：日文…

In [ ]:
from gtts import gTTS
from IPython.display import Audio, display
import os
import uuid

def text_to_speech_gtts(text: str, lang: str = 'zh-TW', filename: str = None, autoplay: bool = True):
    """
    將文字轉為語音（gTTS），並在 Notebook 內播放。
    參數：
      - text: 欲轉語音的文字
      - lang: 語言碼（預設 zh-TW）
      - filename: 儲存檔名（.mp3）。未指定則自動產生。
      - autoplay: 是否顯示播放器（自動播放取決於環境設定）
    回傳：輸出的 MP3 檔名（含路徑）
    """
    text = (text or '').strip()
    if not text:
        raise ValueError('text 為空字串。')

    if not filename:
        filename = f"tts_{uuid.uuid4().hex}.mp3"

    tts = gTTS(text=text, lang=lang)
    tts.save(filename)

    if autoplay:
        display(Audio(filename))
    return filename

print('函式 text_to_speech_gtts 已定義。')


函式 text_to_speech_gtts 已定義。


## 3) 快速體驗：輸入一句話並播放
把 `text` 改成你要唸的內容，執行後就會看到播放器並可播放。

In [ ]:
text = "大家早安，今天上課內容是拉氏轉換。"  # ← 在這裡輸入你想要的文字
lang = 'zh-TW'  # 可改為 'zh-CN'、'en'、'ja'...
out_file = text_to_speech_gtts(text, lang=lang)
print('已輸出：', out_file)


已輸出： tts_33672013994f45459a5230a0e73dc7b0.mp3


## 4) 互動式輸入（若有 `ipywidgets`）
以下單元會偵測是否有 `ipywidgets`。
如果有，就提供簡易的文字框與語言選單；若無，則會提示你改用上方的變數輸入方式。

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    lang_dd = widgets.Dropdown(options=['zh-TW','zh-CN','en','ja','ko','fr','de','es'], value='zh-TW', description='語言')
    ta = widgets.Textarea(value='請輸入要轉語音的文字...', description='文字', layout=widgets.Layout(width='100%', height='100px'))
    btn = widgets.Button(description='合成並播放', button_style='primary')
    out = widgets.Output()

    def _on_click(_):
        with out:
            clear_output()
            try:
                filepath = text_to_speech_gtts(ta.value, lang=lang_dd.value, autoplay=True)
                print('已輸出：', filepath)
            except Exception as e:
                print('發生錯誤：', e)

    btn.on_click(_on_click)
    display(lang_dd, ta, btn, out)
except Exception as e:
    print('未安裝 ipywidgets，請改用上方的「快速體驗」格子直接指定 text / lang 後執行。')


Dropdown(description='語言', options=('zh-TW', 'zh-CN', 'en', 'ja', 'ko', 'fr', 'de', 'es'), value='zh-TW')

Textarea(value='請輸入要轉語音的文字...', description='文字', layout=Layout(height='100px', width='100%'))

Button(button_style='primary', description='合成並播放', style=ButtonStyle())

Output()

## 5) 批次處理：多段文字 → 多個 MP3
如果你有多句台詞或教學重點，可以一次轉出多個檔案，檔名會依序編號。

In [ ]:
texts = [
    '同學們好，今天我們來複習上一堂課的內容。',
    '接下來會示範如何使用拉氏轉換解微分方程。',
    '請把作業上傳到教學平台，截止時間是本週五晚上十點。'
]
lang = 'zh-TW'

saved_files = []
for i, t in enumerate(texts, start=1):
    fname = f"batch_{i:02d}.mp3"
    text_to_speech_gtts(t, lang=lang, filename=fname, autoplay=False)
    saved_files.append(fname)
saved_files


['batch_01.mp3', 'batch_02.mp3', 'batch_03.mp3']

## 6)（選配）儲存到雲端硬碟（以 Google Colab 為例）
若你使用 **Google Colab** 並希望把產生的 MP3 存進 Google Drive：
1. 執行下方區塊掛載雲端硬碟
2. 將輸出檔名改成 `/content/drive/MyDrive/你的資料夾/xxx.mp3`

> 本步驟 **只在 Colab 需要**。一般 Jupyter 直接存到目前工作目錄即可。

In [ ]:
# 在 Google Colab 可用（本機 Jupyter 不需要）
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    print('已掛載 Google Drive。')
except Exception:
    print('非 Colab 環境，略過掛載。')


非 Colab 環境，略過掛載。


## 7)（說明）離線引擎 `pyttsx3` 的限制
- `pyttsx3` 依賴系統的 TTS 引擎與音訊裝置，雲端環境常缺少這些元件而導致**無法播放或不易輸出**。
- 若你在**本機端**（Windows/macOS/Linux）需要**離線**與**即時播放**，可改用 `pyttsx3`；但在雲端或容器環境可能需要額外安裝系統語音包與音訊驅動。

下面提供參考函式（不建議在 Colab 執行）：

In [ ]:
def text_to_speech_offline_pyttsx3(text: str, rate: int = 180, volume: float = 1.0, filename: str = None):
    """僅供本機參考：使用 pyttsx3 將文字轉為語音。
    在雲端環境可能無法正常播放或輸出。"""
    try:
        import pyttsx3
    except Exception as e:
        raise RuntimeError('缺少 pyttsx3，請先 pip install pyttsx3（建議本機端）。') from e

    engine = pyttsx3.init()
    engine.setProperty('rate', int(rate))
    engine.setProperty('volume', float(volume))
    if filename:
        engine.save_to_file(text, filename)
        engine.runAndWait()
        return filename
    else:
        engine.say(text)
        engine.runAndWait()
        return ''

print('離線函式（pyttsx3）已提供參考。建議僅在本機端使用。')


---
### 小結
- 在雲端環境，**gTTS + Notebook Audio** 是最穩定、最直覺的方案。
- 若要大量製作音檔，可用**批次處理**段落一次轉出多個 MP3。
- 需要 GUI、參數化面板或更進階的專案結構，也可以在此 Notebook 基礎上擴充。

祝你使用順利！

#我問：可以將一段文字轉成一部生動的影片嗎？

可以，但這取決於你想要的影片類型與細節程度。這裡簡單分幾種情境說明：

---

### 🎬 1. **純文字→旁白影片（TTS + 圖像或字幕）**

這類最容易實現，可以結合：

* **語音生成**：用我前面幫你寫的文字轉語音程式（`gTTS` 或 `pyttsx3`）生成旁白。
* **影片生成**：

  * 使用 Python 的 `moviepy` 將背景圖片、字幕與語音合成影片。
  * 或者用 Canva、CapCut、Pika Labs 等工具直接將語音 + 文本自動生成動畫影片。

✅ 適合：

* 教學內容（如你課程中的「看動漫學物理」）
* 解說影片、旁白簡報、教育短片

---

### 🧠 2. **AI 自動生成動畫影片**

若你希望輸入一段劇本文字，AI 幫你自動生成角色、場景、配音與鏡頭運動，這需要使用 AI 影片生成平台，例如：

* **Pika Labs**（[https://pika.art/）：輸入文字提示可產生動態影片。](https://pika.art/）：輸入文字提示可產生動態影片。)
* **Runway ML Gen-2**（[https://runwayml.com/）：支援文字到影片生成。](https://runwayml.com/）：支援文字到影片生成。)
* **Kaiber、Synthesia、HeyGen**：可生成角色口說台詞的動畫講者影片。
* **Animoto、Lumen5**：將長文字自動配圖配音成解說短片。

✅ 適合：

* 想快速將一段文字轉成生動畫面（例如一位老師講課的 AI 影片）
* 想把故事或課文自動畫面化

---

### 💻 3. **Python 自動生成簡易動畫影片（範例）**

這是一個最簡單的程式範例（可在 Jupyter Notebook 執行）：

```python
from moviepy.editor import *
from gtts import gTTS

text = "今天我們要學習拉氏轉換，它能將微分方程轉換成代數問題。"
tts = gTTS(text, lang='zh-TW')
tts.save("voice.mp3")

# 建立背景圖片與文字
clip = ImageClip("background.jpg", duration=8)
txt = TextClip(text, fontsize=40, color='white', method='caption', size=(1280,720)).set_duration(8).set_position('center')

# 合成語音與字幕
audioclip = AudioFileClip("voice.mp3")
final = CompositeVideoClip([clip, txt])
final = final.set_audio(audioclip)
final.write_videofile("output.mp4", fps=24)
```

> 若沒有背景圖，可用 `ColorClip((1280,720), color=(0,0,128))` 代替。

---

我可以幫你：

1. 🎓 **把文字直接變成可執行的 Notebook（生成語音 + 加字幕 + 輸出影片）**，
2. 🎥 或**整合外部 AI 平台（如 Pika Labs）生成動畫影片的自動腳本**。

你希望哪一種？（簡報型旁白影片、動畫故事影片、還是角色講話影片？）
